In [0]:
import tensorflow as tf

In [0]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout,Flatten,Conv2D,MaxPool2D,Dense
from tensorflow.keras.preprocessing import image

In [8]:
! git clone https://github.com/AnnaFilipov93/flowers-analyzer

Cloning into 'flowers-analyzer'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 4404 (delta 8), reused 30 (delta 5), pack-reused 4369
Receiving objects: 100% (4404/4404), 368.70 MiB | 45.58 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Checking out files: 100% (8710/8710), done.


In [0]:
#prepering the data
image_gen=ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                             height_shift_range=0.1, rescale=1/255,
                            shear_range=0.2, zoom_range=0.2,
                            horizontal_flip=True, fill_mode='nearest')


In [6]:
###~~~building the model~~~###

model=Sequential()

#conv layer I
model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(240,240,3),
                 activation='relu'))
#pooling layer I
model.add(MaxPool2D(pool_size=(2,2)))

#conv layer II
model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=(240,240,3),
                 activation='relu'))
#pooling layer II
model.add(MaxPool2D(pool_size=(2,2)))

#conv layer III
model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=(240,240,3),
                 activation='relu'))
#pooling layer III
model.add(MaxPool2D(pool_size=(2,2)))

#conv layer IV
model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=(240,240,3),
                 activation='relu'))
#pooling layer IV
model.add(MaxPool2D(pool_size=(2,2)))

#conv layer V
model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=(240,240,3),
                 activation='relu'))
#pooling layer V
model.add(MaxPool2D(pool_size=(2,2)))

#conv layer VI
model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=(240,240,3),
                 activation='relu'))
#pooling layer VI
model.add(MaxPool2D(pool_size=(2,2)))

#flatten
model.add(Flatten())

#Dense layer
model.add(Dense(128,activation='relu'))

#dropout layer
model.add(Dropout(0.5))

#output layer
model.add(Dense(5,activation='sigmoid'))

model.summary()

#compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 238, 238, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 119, 119, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 117, 117, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 58, 58, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 64)        3

In [14]:
##~~reading the data ~~##
train_dir=os.path.join(os.getcwd(), 'flowers-analyzer/flowers/FLOWERS/train')
test_dir=os.path.join(os.getcwd(), 'flowers-analyzer/flowers/FLOWERS/test')

train_img=image_gen.flow_from_directory(train_dir,
                                         target_size=(240,240), batch_size=32,
                                       class_mode='categorical')


test_img=image_gen.flow_from_directory(test_dir,
                                       target_size=(240,240), batch_size=16, 
                                        class_mode='categorical')


Found 7002 images belonging to 5 classes.
Found 850 images belonging to 5 classes.


In [0]:
###~~~training the model~~~###

results=model.fit_generator(train_img,epochs=150, steps_per_epoch=219,
                            validation_data=test_img, validation_steps=54)

Epoch 1/150
219/219 [==============================] - 112s 510ms/step - loss: 0.7703 - accuracy: 0.7121 - val_loss: 0.8268 - val_accuracy: 0.6882
Epoch 2/150
219/219 [==============================] - 112s 512ms/step - loss: 0.7208 - accuracy: 0.7331 - val_loss: 0.8598 - val_accuracy: 0.6812
Epoch 3/150
219/219 [==============================] - 112s 513ms/step - loss: 0.6911 - accuracy: 0.7379 - val_loss: 0.7929 - val_accuracy: 0.7000
Epoch 4/150
219/219 [==============================] - 112s 512ms/step - loss: 0.6833 - accuracy: 0.7441 - val_loss: 0.7967 - val_accuracy: 0.7224
Epoch 5/150
219/219 [==============================] - 113s 514ms/step - loss: 0.6558 - accuracy: 0.7531 - val_loss: 0.9407 - val_accuracy: 0.6659
Epoch 6/150
219/219 [==============================] - 112s 513ms/step - loss: 0.6441 - accuracy: 0.7552 - val_loss: 0.7871 - val_accuracy: 0.7000
Epoch 7/150
219/219 [==============================] - 113s 514ms/step - loss: 0.6229 - accuracy: 0.7658 - val_loss: 0

In [0]:
model.save('five_flowers_model.h5')

In [0]:
def pred (p, pred_acc):
    if pred_acc[0,p]<0.7:
        print("the model couldn't recognize it")
    elif p==0:
        print("the model recognized it as a daisy")
    elif p==1:
         print("the model recognized it as a dandelion")
    elif p==2:
          print("the model recognized it as a rose")
    elif p==3:
          print("the model recognized it as a sunflower")       
    elif p==4:
          print("the model recognized it as a tulip")     
            

In [0]:
###~~~testing on the tests images~~~###

examples = ['FLOWERS//test//daisy/144076848_57e1d662e3_m.jpg',
            'FLOWERS//test//dandelion/14740350060_a489d9fa06.jpg',
            'FLOWERS//test/tulip/5633266048_4f4bfb2cf1_n.jpg',
            'FLOWERS//test/sunflower/21728822928_9f6817325a_n.jpg',
            'flowers//FLOWERS//test//rose/random_flower.jpg']



In [0]:
def predict_flower(path):
  flower=cv2.imread(path)
  flower=cv2.cvtColor(flower,cv2.COLOR_BGR2RGB)
  plt.imshow(flower)

  flower_file=path
  flower_img=image.load_img(flower_file, target_size=(240,240))
  flower_img=image.img_to_array(flower_img)
  flower_img=np.expand_dims(flower_img,axis=0)
  flower_img=flower_img/255

  p=model.predict_classes(flower_img)
  pred(p)

In [0]:
model=load_model('five_flowers_model.h5')


---### This is a single use script used to manipulate existing data and created randomly altered new images by the means of rotation, adding noise, smoothening the image and creating mirror images. ###---

In [9]:
import numpy as np
import glob
import os
import cv2
import random
from PIL import Image
from skimage.util import random_noise
from zipfile import ZipFile

# extra script to augment data (!!! One time use !!!)

PATH_TO_IMAGES = 'flowers-analyzer/flowers/FLOWERS/train/'
PREFIX = '/augmented_'
  
sub_directories = os.listdir(PATH_TO_IMAGES)

for sub_dir in sub_directories:
  sub_dir_path = PATH_TO_IMAGES + sub_dir
  images = [cv2.imread(file) for file in glob.glob(sub_dir_path + '/*.jpg')]
  aug_images = []

  for im in images:

    rand = random.randint(1, 5)
    augmented_image = None

    if rand == 1:
      angle= random.randint(0,90)
      im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
      im_pil = Image.fromarray(im)
      rotated = im_pil.rotate(-angle)
      augmented_image = np.asarray(rotated)
        
    elif rand == 2:
      angle= random.randint(0,90)
      im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
      im_pil = Image.fromarray(im)
      rotated = im_pil.rotate(angle)
      augmented_image = np.asarray(rotated)

    elif rand == 3:
      augmented_image = np.fliplr(im)

    elif rand == 4:
      noise_img = random_noise(im, mode='s&p',amount=0.005)
      augmented_image = np.array(255*noise_img, dtype = 'uint8')

    else:
      augmented_image = cv2.GaussianBlur(im, (9,9), cv2.BORDER_DEFAULT)

    aug_images.append(augmented_image)

  counter = 0
  for image in aug_images:
    img_name = sub_dir_path + PREFIX + str(counter) + '.jpg'
    cv2.imwrite(img_name, image)
    counter = counter + 1

'''with ZipFile('flowers.zip', 'w') as zipObj:

   for folderName, subfolders, filenames in os.walk(PATH_TO_IMAGES):

       for filename in filenames:
           #create complete filepath of file in directory
           filePath = os.path.join(folderName, filename)
           # Add file to zip
           zipObj.write(filePath)
'''
print('Augmented images created...')

Augmented images created...
